# <u>**Streaming Service Comparison**</u>

### **Objective:**
- Determine which streaming platform hosts the majority of content I enjoy so that I can pare down the services to which I subscribe. 

### **Data Sources:**
- [Netflix via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset)
- [Hulu via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-hulu-dataset)
- [Prime via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-amazon-prime-dataset/data)
- [AppleTV via Kaggle.com](https://www.kaggle.com/datasets/octopusteam/full-apple-tv-dataset)

# <u>**Data Collection & Loading**</U>

## **Import Pandas, Numpy, Matplotlib, Wordcloud, and PIL**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wordcloud as wc
from PIL import Image

## **Data Load**

In [8]:
# Assign dataset names

list_of_names = ['Netflix','Hulu','Prime','AppleTV']

combined_list = []

for i in range(len(list_of_names)):
    temp_df = pd.read_csv(list_of_names[i]+".csv")
    combined_list.append(temp_df)